In [74]:
import os 
import pandas as pd
import subprocess
import seaborn as sns

In [100]:
def function2(data):
    import pandas as pd 
    import pickle 
    # assign col names to the table 
    data.columns =['Strain_ID', 'Genome_length', 'Ratio_1kbp_bins_covered', 'Ratio_GC-content_uncovered_bins', 'Ratio_N-content_uncovered_bins', 'Reads_mapped', 'Total_reads']
    # store strains ID into another column 
    strains = data['Strain_ID']
    # drop strains ID col from the table in order to correctly apply the model  
    data = data.drop(['Strain_ID'], axis=1)
    # import the 2 models 
    final_model = pickle.load(open('real_final_model.sav', 'rb')) 
    scaler = pickle.load(open('scaler.sav', 'rb'))
    # normalize data 
    data = scaler.transform(data)
    data = pd.DataFrame(data, columns = ['Genome_length', 'Ratio_1kbp_bins_covered', 'Ratio_GC-content_uncovered_bins', 'Ratio_N-content_uncovered_bins', 'Reads_mapped', 'Total_reads'])
    # predict using the final model 
    pred = final_model.predict(data)
    pred = pd.DataFrame(pred, columns = ['Class'], index = None)
    # merge the outputs 
    data2 = [strains, data, pred]
    data2 = pd.concat(data2, axis=1)
    return data2 

In [92]:
# testing the function 
data = pd.read_excel('final.xlsm', header=None)
data.columns =['Strain_ID', 'Genome_length', 'Ratio_1kbp_bins_covered', 'Ratio_GC-content_uncovered_bins', 'Ratio_N-content_uncovered_bins', 'Reads_mapped', 'Total_reads', 'Class']
data = data.drop(['Class'], axis = 1)
data.shape
data.head()

,Strain_ID,Genome_length,Ratio_1kbp_bins_covered,Ratio_GC-content_uncovered_bins,Ratio_N-content_uncovered_bins,Reads_mapped,Total_reads
0,293,6771240,0.000295,0.673722,0.000115,2,4506372
1,294,31689102,0.000000,0.596989,0.014204,0,4506372
2,300,5773548,0.000000,0.624336,0.000000,0,4506372
3,303,19957976,0.000000,0.610632,0.000218,0,4506372
4,316,9256019,0.000000,0.622664,0.000000,0,4506372


In [101]:
pred = function2(data)
pred

,Strain_ID,Genome_length,Ratio_1kbp_bins_covered,Ratio_GC-content_uncovered_bins,Ratio_N-content_uncovered_bins,Reads_mapped,Total_reads,Class
0,293,0.228558,-0.219520,1.078705,-0.168407,-0.117813,-0.420667,0
1,294,4.039962,-0.220521,0.485007,-0.154483,-0.117818,-0.420667,0
2,300,0.075952,-0.220521,0.696596,-0.168520,-0.117818,-0.420667,0
3,303,2.245584,-0.220521,0.590566,-0.168305,-0.117818,-0.420667,0
4,316,0.608627,-0.220521,0.683660,-0.168520,-0.117818,-0.420667,0
...,...,...,...,...,...,...,...,...
3138,RuminococcusC4,-0.173045,3.080917,-1.227358,-0.167632,3.449620,2.124029,0
3139,RuminococcusC5,-0.274968,0.172065,-0.771327,-0.106793,-0.023186,2.124029,1
3140,StreptococcusC1,-0.487998,-0.199688,-1.139100,9.422665,-0.117437,2.124029,0
3141,StreptococcusC2,-0.475110,-0.165879,-1.085164,-0.168268,-0.109377,2.124029,0


In [94]:
pred['Class'].value_counts() 
# 2 strains more 

0    3021
1     122
Name: Class, dtype: int64

In [95]:
pred.groupby('Class').mean() # quality check 

,Genome_length,Ratio_1kbp_bins_covered,Ratio_GC-content_uncovered_bins,Ratio_N-content_uncovered_bins,Reads_mapped,Total_reads
Class,,,,,,
0,-0.016171,-0.063214,0.017652,0.003170,-0.062128,-0.006513
1,0.400427,1.565314,-0.437108,-0.078491,1.538438,0.161287
